## Welcome to your notebook.


#### Run this cell to connect to your GIS and get started:

In [ ]:
from arcgis.gis import GIS
import requests
import pandas as pd
import json
from arcgis.features import FeatureLayerCollection
gis = GIS("home")

In [6]:
url = "https://cdn5.editmysite.com/app/store/api/v28/editor/users/150158922/sites/162638533544758490/products?page=1&per_page=200&sort_by=category_order&sort_order=asc&categories[]=DMADMXTG2OFB7ICIEFDHBBP7&include=images,media_files,discounts&excluded_fulfillment=dine_in&cache-version=20"
response = requests.get(url)
data = response.json()

file_name = "StreetSignInventory.json"
with open(file_name, "w") as file:
    json.dump(data, file, indent=4)
    
with open(file_name, "r") as file:
    data_from_file = json.load(file)
    
inventory = data_from_file['data']

df = pd.json_normalize(inventory)

pd.options.display.max_columns=300

In [7]:
df['Quantity'] = df['inventory.total']
df['Street Sign Name'] = df['name'].str.split(' Street Sign').str[0]
df['Site Link'] = "https://lowerpaxtontownship.square.site/" + df['site_link']
df['Creation Date'] = df['created_date']
df['Last Updated Date'] = df['updated_date']
subset_df = df[['Street Sign Name', 'Quantity', 'Creation Date', 'Last Updated Date', 'Site Link']]
subset_df.to_csv('StreetSignInventory5.csv')

In [8]:
#Publishing to AGOL Code
csv_file_path = '/arcgis/StreetSignInventory5.csv'
csv_file_name = 'StreetSignInventory5.csv'

# item_properties = {
#     "title": "StreetSignInventory_SquareAPI",
#     "description": "This table is created from a Jupyter Notebook with Python from the Lower Paxton Township Square Site API",
#     "type": "CSV"
# }

# csv_item = gis.content.add(item_properties, csv_file_path)
# csv_item_service = csv_item.publish()


In [10]:
hosted_table_item = gis.content.get("b94a091c0f6d42beb64f00366f973bf5")

flc = FeatureLayerCollection.fromitem(hosted_table_item)

flc.manager.overwrite(csv_file_name)

print("Table updated successfully!")

Table updated successfully!
